# Muon Sampling Histogram Generation

In order to sample from histograms with FLUKA's built in `sample_histogram_momentum_energy()` function, we must generate a histogram file with the following format:

`#Emin   Emax    dN/dE`<br>
`#--------------------`<br>
 `10.  20.  11.`
 
 The bin minima in the leftmost column, the maxima in the middle, and the relative count in the final, rightmost column

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

## ENERGY

$$\frac{dN}{dE_\mu} = Ae^{-bh(\gamma_\mu -1)}\cdot(E_\mu + \epsilon_\mu(1-e^{-bh}))^{-\gamma_\mu}$$

In [3]:
def mh_energy_probs(energies, zenith = 0, sample = True):

    SNOLAB_DEPTH = 5.89 #km.w.e
    
    b = 0.4 #km.w.e^{-1}
    epsilon = 693 #GeV
    gamma = 3.77
    h = (SNOLAB_DEPTH/np.cos(zenith))
    
    prob = lambda E : np.exp(-b*h*(gamma - 1))*(E + epsilon*(1 - np.exp(-b*h)))**(-gamma)
    norm_const = quad(prob, 0, np.pi/2)[0]
    if not sample:
        array = prob(energies)/norm_const
    else:
        array = prob(energies)/np.sum(prob(energies))

    return array

In [ ]:
def make_energy_file(e_min, e_max, N, filename = 'mu_e_hist.txt'):

    energies = np.linspace(e_min, e_max, N)
    probs = mh_energy_probs(energies)
    hist = np.histogram(energies, weights=probs, bins = N)
    file = open(filename, 'w')

    filestring = "%.3g    %.3g    %.3g" %(0, energies[0], hist[0][0]) + "\n"
    file.write(filestring)
    for i in range(N - 1):
        filestring = "%.3g    %.3g    %.3g" %(energies[i], energies[i+1], hist[0][i]) + "\n"
        file.write(filestring)
        
    file.close()

In [1]:
def make_zenith_file(N, filename = 'mu_zen_hist.txt'):
    radius = 6.1722
    height = 12.8
    gen_offset = 15

    gen_radius = np.tan(1)*(height + gen_offset) + radius

    slantDepth = 5.89 #km w.e.
    I1 = 8.60e-6 #  0.53e-6 /sec/cm^2/sr
    I2 = 0.44e-6 #  0.06e-6 /sec/cm^2/sr
    lam1 = 0.45 #  0.01 km.w.e.
    lam2 = 0.87 #  0.02 km.w.e.

    #Zenith angle range
    cosine = np.linspace(0.1, 1, N)

    #Muon Angular distribution intensity
    meiHime = (I1*np.exp(-slantDepth/(lam1*cosine))+I2*np.exp(-slantDepth/(lam2*cosine)))*np.sin(np.arccos(cosine))
    meiHime = meiHime/np.sum(meiHime)

    file = open(filename, 'w')

    for i in range(len(meiHime)-1):
        filestring = "%.3g    %.3g    %.3g" %(cosine[i], cosine[i+1], meiHime[i]) + "\n"
        file.write(filestring)

    file.close()

In [53]:
make_zenith_file(20)

# Making phase space file for FLUKA
```
! 8.2. File format:
      ! The phase space file has to contain the following columns in this order:

      ! - Particle code [integer]

      ! - Particle momentum / energy [double precision]

      ! - Starting X coordinate [double precision]
      ! - Starting Y coordinate [double precision]
      ! - Starting Z coordinate [double precision]

      ! - Starting X direction cosine [double precision]
      ! - Starting Y direction cosine [double precision]
      ! - Starting Z direction cosine [double precision]

      ! - Particle weight [double precision]```

In [1]:
def make_phase_space_file(num_muons, roi_radius = 0, roi_height = 0, filename = 'muon_file.txt'):
    from random import random
    import muon_functions as mf

    if roi_radius == 0:
        roi_radius = mf.OD_RADIUS + 1
        roi_height = mf.OD_HEIGHT + 1

    roi = mf.OuterDetector(roi_radius, roi_height)

    file_stream = open(filename, 'w')

    muarray = mf.intersecting_muons(num_muons, roi)

    for muon in muarray:
        particle_code = 10
    
        if random() > 0.5:
            particle_code = 11
            
        file_stream.write(str(particle_code) + ' ' + str(muon) + '\n')

    file_stream.close()
        

In [3]:
make_phase_space_file(5000)

In [1]:
def make_vertical_muons(num_per_interval = 2, start = 0, stop = 12, increment = 0.01, energy = 360, init_z = 22, filename = 'vertical_muons.txt'):
    from random import random
    import muon_functions as mf
    import numpy as np

    file_stream = open(filename, 'w')

    iterations = int((stop - start)/increment)

    for i in range(iterations):

        init_x = start + i*increment

        muon = mf.Muon(0, 0, initial = (init_x, 0, init_z))

        for j in range(num_per_interval):
            ramdom = random()
            if ramdom > 0.5:
                particle_code = 11
            else:
                particle_code = 10 

            file_stream.write(str(particle_code) + ' ' + str(muon) + '\n')

    file_stream.close()



In [25]:
make_vertical_muons()